## HW 6: DS 340H
Research question: Is there any correlation between the number of Bluebike rides and their average duration and the number of Commuter Rail rides in a sample of Bluebike and Computer Rail rides from July 2018-January 2025?

In [40]:
import nbconvert
from datetime import datetime
import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path


In [3]:
with open('mbta.csv','r',newline='') as file:
    data = csv.DictReader(file)
    mbta = pd.DataFrame(data)

mbta.drop(['daycount', 'ridership_average', 'ObjectId'], axis=1, inplace=True)
mbta.rename(columns={'\ufeffmonth_of_service':'month_of_service'}, inplace=True)
# mbta['year'] = mbta['month'].apply(lambda x: x[0:4])

# mbta['month'] = mbta['month'].apply(lambda x: x[5:7])
mbta.head()

,month_of_service,daytype,route_or_line,ridership_total
0,2024/12/01 05:00:00+00,Weekday,Blue Line,905115
1,2024/12/01 05:00:00+00,Weekday,Orange Line,2208993
2,2024/12/01 05:00:00+00,Weekday,Red Line,2367621
3,2024/12/01 05:00:00+00,Weekday,Heavy Rail,5481728
4,2024/12/01 05:00:00+00,Weekday,Green Line,1670698


Filtering selected mbta lines:

In [ ]:
selected_lines = ['Blue Line', 'Orange Line', 'Red Line', 'Green Line']

# Filter the dataset based on the 'route_or_line' column
filtered_mbta = mbta[mbta['route_or_line'].isin(selected_lines)]

Selecting only the total values for the daytype, and eliminating the daytype as a variable:

In [14]:
mbta_final = filtered_mbta[filtered_mbta['daytype'] == 'Total']
mbta_final.drop('daytype', axis=1, inplace=True)


/var/folders/8g/pmfcwbg561xgw6b04jzwczp00000gn/T/ipykernel_64443/2115895493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbta_final.drop('daytype', axis=1, inplace=True)


Code to pivot the MBTA dataset so that each line has its own column:

In [17]:
wide_df = mbta_final.pivot(index='month_of_service', columns='route_or_line', values='ridership_total')
wide_df.columns = [f"{line.lower().replace(' ', '_')}_riders" for line in wide_df.columns]
wide_df = wide_df.reset_index()

Gathers the total ride counts and average trip duration from the bluebike files:

In [57]:
bluebike = pd.DataFrame(columns=["month", "year", "rides_total", "avg_ride_length"])

months = []
years = []
rideCounts = []
avgDurations = []

filePathObj = Path('bluebike')
for fileObj in filePathObj.iterdir():
    if fileObj.is_file():
        with open(fileObj, 'r') as file:
            
            year = fileObj.name[0:4]
            month = fileObj.name[4:6]

            df = pd.read_csv(fileObj)
            
            if 'starttime' in df.columns and 'stoptime' in df.columns:
                start_col = 'starttime'
                end_col = 'stoptime'
            elif 'started_at' in df.columns and 'ended_at' in df.columns:
                start_col = 'started_at'
                end_col = 'ended_at'
            else:
            # Skip the CSV if neither of the expected column pairs is found
                print(f"Skipping {fileObj} as it doesn't have the expected columns.")
                continue

            df[start_col] = pd.to_datetime(df[start_col])
            df[end_col] = pd.to_datetime(df[end_col])

            df['duration'] = df[end_col] - df[start_col]

            avgDurations.append(df['duration'].mean())
            rideCounts.append(len(df))
            months.append(month)
            years.append(year)

bluebike["rides_total"] = rideCounts
bluebike["month"] = months
bluebike["year"] = years
bluebike["avg_ride_length"] = avgDurations
bluebike.head()


Using 'started_at' and 'ended_at' columns in 202407-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202501-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202310-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202409-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202403-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202404-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202405-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202402-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202408-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202312-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202311-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202401-bluebikes-tripdata.csv
Using 'started_at' and 'ended_at' columns in 202406-bluebikes-tripdata.csv
Using 'started_at' and 'e

,month,year,rides_total,avg_ride_length
0,07,2024,542622,0 days 00:19:15.277085820
1,01,2025,162316,0 days 00:12:47.080784254
2,11,2022,290621,0 days 00:21:47.530517429
3,07,2020,259726,0 days 00:38:40.519126067
4,01,2021,71805,0 days 00:20:20.219710354


Converts time for the ride lengths into a more legible format:

In [58]:
def makeLegibleTime(td):
    total_seconds = td.total_seconds()
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{int(hours):02}:{int(minutes):02}:{seconds:05.2f}"

bluebike['avg_ride_length'] = pd.to_timedelta(bluebike['avg_ride_length'])
bluebike['avg_ride_length'] = bluebike['avg_ride_length'].apply(makeLegibleTime)
bluebike['avg_ride_length'] = pd.to_timedelta(bluebike['avg_ride_length']).dt.total_seconds()

bluebike.head()

,month,year,rides_total,avg_ride_length
0,07,2024,542622,1155.28
1,01,2025,162316,767.08
2,11,2022,290621,1307.53
3,07,2020,259726,2320.52
4,01,2021,71805,1220.22


Converting the months and years into datetime objects, and reorganizing the data frame so that each row is a given date:

In [59]:
mbta['month_year'] = pd.to_datetime(mbta[['year', 'month']].assign(day=1))
bluebike['month_year'] = pd.to_datetime(bluebike[['year', 'month']].assign(day=1))

In [ ]:

bluebike['month_of_service'] = pd.to_datetime(bluebike[['year', 'month']].assign(day=1))

bluebike.drop(['month', 'year', 'month_year'], axis=1, inplace=True)
bluebike = bluebike.rename(columns={'rides_total': 'bluebike_riders',
                                    'avg_ride_length': 'avg_bluebike_length'})

col_order = ['month_of_service', 'bluebike_riders', 'avg_bluebike_length']
bluebike = bluebike[col_order]
bluebike.to_csv('bluebike.csv', index=False)


In [34]:
wide_df['month_of_service'] = pd.to_datetime(wide_df['month_of_service']).dt.date
bluebike['month_of_service'] = pd.to_datetime(bluebike['month_of_service']).dt.date

In [55]:
merged_df = pd.merge(wide_df, bluebike, on='month_of_service')
merged_df['month_of_service'] = pd.to_datetime(merged_df['month_of_service'])

Adding a column to indicate when subsidies were introduced:

In [57]:
subsidy = datetime(2023, 10, 1).date()
subsidy = pd.to_datetime(subsidy)
merged_df['year'] = merged_df['month_of_service'].dt.year
merged_df['bluebike_subsidy'] = (merged_df['month_of_service'] >= subsidy).astype(int)

merged_df.to_csv('merged_mbta_bluebike.csv',index=False)